In [2]:
import pandas as pd

# Replace 'your_file.xlsx' with the path to your Excel file
file_path = '../../storage/Judicial/raw_data_juicios.xlsx'

# Read the Excel file
df = pd.read_excel(file_path)

# print head
print(df.head())

        Cédula        id_Juicio                                  Delito  \
0  100004407.0  03204201700015               COBRO DE PAGARÉ A LA ORDEN   
1  100007269.0  01204201905175                                ALIMENTOS   
2  100007269.0  01204201903452                                ALIMENTOS   
3  100007269.0  01204201900368   INVESTIGACIÓN DE PATERNIDAD/MATERNIDAD   
4  100013465.0  01371202300415                PAGO DE HABERES LABORALES   

         Fecha_de_Ingreso        mes     año                 Estado  \
0 2017-01-09 13:23:24.813    January  2017.0              No activo   
1 2019-09-09 15:50:33.447  September  2019.0  No se pudo determinar   
2 2019-06-12 13:19:18.733       June  2019.0              No activo   
3 2019-01-18 21:50:53.770    January  2019.0              No activo   
4 2023-12-05 17:28:11.537   December  2023.0                 Activo   

  Monto_por_juicio  
0              NaN  
1              NaN  
2              NaN  
3              NaN  
4              Na

#### Fist wee need to clasify the juicios types into 5 diffrent catgories: Alimentos, Familiares, Penales, Estafas, Cobros

In [28]:
# get all difrent types of value sin the colum 'Delito'
unique_delitos = df['Delito'].unique()
print(unique_delitos)
print(unique_delitos.shape[0])

['COBRO DE PAGARÉ A LA ORDEN' 'ALIMENTOS'
 'INVESTIGACIÓN DE PATERNIDAD/MATERNIDAD' 'PAGO DE HABERES LABORALES'
 'PRESCRIPCIÓN ADQUISITIVA DE DOMINIO'
 'PRESCRIPCIÓN EXTRAORDINARIA ADQUISITIVA DE DOMINIO' 'LESIONES'
 'AMPARO POSESORIO' 'DERECHOS FUNDAMENTALES DEL CONSUMIDOR'
 'ARCHIVO DE LA INVESTIGACIÓN PREVIA'
 'OPOSICIÓN A LA RENDICIÓN DE CUENTAS' 'COBRO DE DINERO' 'REINVINDICACIÓN'
 'RESOLUCIÓN DE CONTRATO'
 'COBRO DE FACTURAS POR BIENES Y SERVICIOS, HONORARIOS PROFESIONALES CUANDO LA PRETENSIÓN NO SEA EXIGIBLE EN PROCEDIMIENTO MONITORIO O EN VIA EJECUTIVA'
 'TERMINACIÓN DE CONTRATO' 'COBRO DE LETRA DE CAMBIO'
 'CONTRAVENCIONES DE TRÁNSITO' 'CONTRATO DE MUTUO O PRESTAMO'
 'DIVORCIO POR CAUSAL'
 'INCUMPLIMIENTO DE DECISIONES LEGÍTIMAS DE AUTORIDAD COMPETENTE'
 'COBRO DE HONORARIOS DE ABOGADO' 'PAGO POR CONSIGNACIÓN'
 'FACTURAS O DOCUMENTOS ART. 356 NUM.2' 'FACTURA O PRIMA DE SEGUROS'
 'PARTICION NO VOLUNTARIA' 'APLICACIÓN DEL PRINCIPIO DE OPORTUNIDAD'
 'NULIDAD DEL ACTA DE MEDIACIÓN

In [30]:
key_words_with_familiares = ['DIVORCIO','UNIÓN DE HECHO', 'ALIMENTOS', 'PENSIÓN', 'GUARDA', 'CUSTODIA', 'VISITA', 'PATRIA POTESTAD', 'ADOPCIÓN', 'FILIACIÓN', 'PATERNIDAD', 'MATERNIDAD', 'FAMILIAR', 'FAMILIA', 'CONYUGAL', 'CONCUBINATO', 'CONCUBINO', 'CONCUBINA', 'MATRIMONIO', 'CONYUGE', 'CONYUGES' ]

family_delitos = []
filtered_delitos = []
# take out all the delitos from unique_delitos that have the key words found in key_words_with_familiares
for delitos in unique_delitos:
    # if delitos is a string 
    if type(delitos) == str:
        # for every worrd in delitos check if it is in key_words_with_familiares
        for word in delitos.split():
            if word in key_words_with_familiares:
                # add delitos to family_delitos
                family_delitos.append(delitos)
                break
        # if delitos is not in family_delitos add it to filtered_delitos
        if delitos not in family_delitos:
            filtered_delitos.append(delitos)
print('family_delitos', len(family_delitos))
print(filtered_delitos)
print(len(filtered_delitos))

        

family_delitos 45
['COBRO DE PAGARÉ A LA ORDEN', 'INVESTIGACIÓN DE PATERNIDAD/MATERNIDAD', 'PAGO DE HABERES LABORALES', 'PRESCRIPCIÓN ADQUISITIVA DE DOMINIO', 'PRESCRIPCIÓN EXTRAORDINARIA ADQUISITIVA DE DOMINIO', 'LESIONES', 'AMPARO POSESORIO', 'DERECHOS FUNDAMENTALES DEL CONSUMIDOR', 'ARCHIVO DE LA INVESTIGACIÓN PREVIA', 'OPOSICIÓN A LA RENDICIÓN DE CUENTAS', 'COBRO DE DINERO', 'REINVINDICACIÓN', 'RESOLUCIÓN DE CONTRATO', 'COBRO DE FACTURAS POR BIENES Y SERVICIOS, HONORARIOS PROFESIONALES CUANDO LA PRETENSIÓN NO SEA EXIGIBLE EN PROCEDIMIENTO MONITORIO O EN VIA EJECUTIVA', 'TERMINACIÓN DE CONTRATO', 'COBRO DE LETRA DE CAMBIO', 'CONTRAVENCIONES DE TRÁNSITO', 'CONTRATO DE MUTUO O PRESTAMO', 'INCUMPLIMIENTO DE DECISIONES LEGÍTIMAS DE AUTORIDAD COMPETENTE', 'COBRO DE HONORARIOS DE ABOGADO', 'PAGO POR CONSIGNACIÓN', 'FACTURAS O DOCUMENTOS ART. 356 NUM.2', 'FACTURA O PRIMA DE SEGUROS', 'PARTICION NO VOLUNTARIA', 'APLICACIÓN DEL PRINCIPIO DE OPORTUNIDAD', 'NULIDAD DEL ACTA DE MEDIACIÓN', 'REC

In [ ]:
# get all rows with 'Delito' in key_words_with_familiares
df_familiares = df[df['Delito'].isin(key_words_with_familiares)]
print(df_familiares.head())
print(df_familiares.shape[0])

In [22]:
# change to strign in df
df['Cédula'] = df['Cédula'].astype(str)

In [25]:
# get only the cedula column
cedula_comun = df['Cédula']

# for each cedula get the number of time it has bee repeated
cedula_counts = cedula_comun.value_counts()
# print the number of times
print(cedula_counts)

Cédula
9.284309e+08    99
1.400412e+09    88
7.046659e+08    87
1.400513e+09    86
1.400286e+09    86
                ..
9.422024e+08     1
9.422002e+08     1
9.422000e+08     1
9.421969e+08     1
1.104264e+09     1
Name: count, Length: 512903, dtype: int64


In [26]:
# add the cedula_counts back to the original data frame
df = pd.merge(df, cedula_counts, left_on='Cédula', right_index=True, how='left')

print(df.head())

        Cédula        id_Juicio                                  Delito  \
0  100004407.0  03204201700015               COBRO DE PAGARÉ A LA ORDEN   
1  100007269.0  01204201905175                                ALIMENTOS   
2  100007269.0  01204201903452                                ALIMENTOS   
3  100007269.0  01204201900368   INVESTIGACIÓN DE PATERNIDAD/MATERNIDAD   
4  100013465.0  01371202300415                PAGO DE HABERES LABORALES   

         Fecha_de_Ingreso        mes     año                 Estado  \
0 2017-01-09 13:23:24.813    January  2017.0              No activo   
1 2019-09-09 15:50:33.447  September  2019.0  No se pudo determinar   
2 2019-06-12 13:19:18.733       June  2019.0              No activo   
3 2019-01-18 21:50:53.770    January  2019.0              No activo   
4 2023-12-05 17:28:11.537   December  2023.0                 Activo   

  Monto_por_juicio  count  
0              NaN    1.0  
1              NaN    3.0  
2              NaN    3.0  
3         